In [1]:
import numpy as np
import pandas as pd
import glob
from astropy.table import Table
import matplotlib.pyplot as plt
import json 
import astropy

In [2]:
response_dict = {
    "#1" : 1,
    "#2" : 2,
    "#3" : 3,
    "#4" : 4,
    "#5" : 5,
    "No good match" : 0,
}

In [3]:
classification = Table.from_pandas(pd.read_csv(r'C:\Users\20xha\Documents\Caltech\Research\ZooniverseClassifications\zwicky-chemical-factory-classifications_all.csv'))

In [4]:
ccs_workflow = np.where(classification["workflow_name"] == "Core Collapse Supernovae")

In [5]:
classification[ccs_workflow][0]

classification_id,user_name,user_id,user_ip,workflow_id,workflow_name,workflow_version,created_at,gold_standard,expert,metadata,annotations,subject_data,subject_ids
int64,str34,float64,str20,int64,str25,float64,str23,float64,float64,str1297,str532,str485,int64
288484944,cfremling,2128769.0,f614d887c5bb9da07e02,16969,Core Collapse Supernovae,8.21,2020-11-20 03:06:17 UTC,--,--,"{""source"":""api"",""session"":""b07e8eff7821c34f6244ef6e4aabc4bce6d598ab906f51424d36f66c90709c84"",""viewport"":{""width"":1822,""height"":861},""started_at"":""2020-11-20T03:05:50.856Z"",""user_agent"":""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36"",""utc_offset"":""28800"",""finished_at"":""2020-11-20T03:06:17.009Z"",""live_project"":true,""interventions"":{""opt_in"":true,""messageShown"":false},""user_language"":""en"",""user_group_ids"":[],""subject_dimensions"":[{""clientWidth"":1280,""clientHeight"":720,""naturalWidth"":4800,""naturalHeight"":2700},{""clientWidth"":1280,""clientHeight"":720,""naturalWidth"":4800,""naturalHeight"":2700},{""clientWidth"":1280,""clientHeight"":720,""naturalWidth"":4800,""naturalHeight"":2700},{""clientWidth"":1280,""clientHeight"":720,""naturalWidth"":4800,""naturalHeight"":2700},{""clientWidth"":1280,""clientHeight"":720,""naturalWidth"":4800,""naturalHeight"":2700}],""subject_selection_state"":{""retired"":false,""selected_at"":""2020-11-20T03:05:50.742Z"",""already_seen"":false,""selection_state"":""normal"",""finished_workflow"":false,""user_has_finished_workflow"":false},""workflow_translation_id"":""39739""}","[{""task"":""T0"",""task_label"":""Which spectrum is the best match?"",""value"":""#1""}]","{""51494375"":{""retired"":null,""z_snid"":""0.436"",""#Image0"":""snidfits_emclip_ZTF17aaazdba_20190330_P60_v1_1.png"",""#Image1"":""snidfits_emclip_ZTF17aaazdba_20190330_P60_v1_2.png"",""#Image2"":""snidfits_emclip_ZTF17aaazdba_20190330_P60_v1_3.png"",""#Image3"":""snidfits_emclip_ZTF17aaazdba_20190330_P60_v1_4.png"",""#Image4"":""snidfits_emclip_ZTF17aaazdba_20190330_P60_v1_5.png"",""!ZTF_Name"":""ZTF17aaazdba"",""z_snid_err"":""0.016"",""!ZTF_Version"":""ZTF17aaazdba_20190330_P60_v1""}}",51494375


In [6]:
subject_ids_unique = np.unique(classification[ccs_workflow]["subject_ids"])

In [7]:
right_workflow = classification[ccs_workflow]

In [8]:
len(subject_ids_unique)

8777

In [9]:
data_np = np.load(r'C:\Users\20xha\Documents\Caltech\Research\ZooniverseClassifications\zwicky-chemical-factory-classifications_all.npy', allow_pickle = True)

In [10]:
len(data_np[:,1][0])

1

In [11]:
data_array = []
for i in data_np:
    if(len(i[1]) > 10):
        data_array.append([i[0], right_workflow[i[1]]])

In [12]:
size = []
for i in data_np[:,1]:
    size.append(len(i))

In [13]:
exec("dict =" + data_array[0][1]["annotations"][0])

In [14]:
json.loads(data_array[0][1]["annotations"][0])[0]

{'task': 'T0',
 'task_label': 'Which spectrum is the best match?',
 'value': 'No good match'}

In [15]:
dict[0]["value"].split("#")[-1]

'No good match'

In [16]:
data_array[0][1]["annotations"]

"[{""task"":""T0"",""task_label"":""Which spectrum is the best match?"",""value"":""No good match""}]"
"[{""task"":""T0"",""task_label"":""Which spectrum is the best match?"",""value"":""#5""}]"
"[{""task"":""T0"",""task_label"":""Which spectrum is the best match?"",""value"":""#5""}]"
"[{""task"":""T0"",""task_label"":""Which spectrum is the best match?"",""value"":""No good match""}]"
"[{""task"":""T0"",""task_label"":""Which spectrum is the best match?"",""value"":null}]"
"[{""task"":""T0"",""task_label"":""Which spectrum is the best match?"",""value"":""#5""}]"
"[{""task"":""T0"",""task_label"":""Which spectrum is the best match?"",""value"":""#4""}]"
"[{""task"":""T0"",""task_label"":""Which spectrum is the best match?"",""value"":""No good match""}]"
"[{""task"":""T0"",""task_label"":""Which spectrum is the best match?"",""value"":""#5""}]"
"[{""task"":""T0"",""task_label"":""Which spectrum is the best match?"",""value"":""No good match""}]"
"[{""task"":""T0"",""task_label"":""Which spectrum is the best match?"",""value"":""No good match""}]"


In [17]:
ResultsTable_Zooniverse = Table(
                    names=("ZTF_Name", "Version", "subject_ids", "NoMatch", "#1", "#2", "#3", "#4", "#5"
                    ),
                    meta={"name": "Spectrum Results after Zooniverse"},
                    dtype=("U64", "U64", "int32", "int32", "int32", "int32", "int32", "int32", "int32"
                          )
                    )
for line in data_array:
    row = []
    data = line[1]
    metadata = json.loads(data[0]["subject_data"])[str(line[0])]
    row.append(metadata["!ZTF_Name"])
    row.append(metadata["!ZTF_Version"] + ".ascii")
    row.append(line[0])
    responses = [0, 0, 0, 0, 0, 0]
    for i in data:
        value = json.loads(i["annotations"])[0]["value"]
        if(value is not None):
            index = response_dict[value]
            responses[index] = responses[index] + 1
    row.extend(responses)
    ResultsTable_Zooniverse.add_row(row)

In [18]:
metadata = json.loads(data[0]["subject_data"])[str(line[0])]
value = json.loads(i["annotations"])[0]["value"]

In [19]:
np.sum(ResultsTable_Zooniverse["NoMatch"])

17874

In [20]:
np.sum(ResultsTable_Zooniverse["#1"])

20759

In [21]:
np.sum(ResultsTable_Zooniverse["#2"])

14892

In [22]:
np.sum(ResultsTable_Zooniverse["#3"])

13749

In [23]:
np.sum(ResultsTable_Zooniverse["#4"])

13530

In [24]:
np.sum(ResultsTable_Zooniverse["#5"])

13431

In [25]:
ResultsTable_Zooniverse.to_pandas().to_csv("FirstAnalysis.csv")

In [26]:
snidoutput = Table.from_pandas(pd.read_csv("SNID_2020_output.csv"))

In [27]:
joined_results = astropy.table.join(snidoutput, ResultsTable_Zooniverse)

In [28]:
ResultsTable_Zooniverse_Classes = Table(
                    names=("ZTF_Name", "Version", "subject_ids", 'Ia', 'Ia-norm', 'Ia-91T', 'Ia-91bg', 'Ia-csm', 'Ia-02cx', 'Ia-03fg',
                           'Ib', 'Ib-norm', 'IIb', 'Ibn', 'Ic', 'Ic-norm', 'Ic-BL', 'Ic-SLSN', 'II', 'II-norm', 'IIn',
                           'NotSN', 'AGN', 'Gal', 'LBV', 'M-star', 'QSO', 'C-star', 'TDE', "NoMatch", "rlap", "z"
                    ),
                    meta={"name": "Vote Types"},
                    dtype=("U64", "U64", "int32", "int32", "int32", "int32", "int32", "int32" , "int32", "int32", "int32", "int32", "int32", "int32",
                           "int32", "int32", "int32", "int32", "int32", "int32", "int32", "int32", "int32", "int32", "int32", "int32", "int32", "int32", "int32", "int32", "float32", "float32"
                          )
                    )

for i in range(len(joined_results)):
    line = joined_results[i]
    row = [line["ZTF_Name"], line["Version"], line["subject_ids"]]
    row.extend(np.zeros(28))
    row.append(line["NoMatch"])
    ResultsTable_Zooniverse_Classes.add_row(row)
    ResultsTable_Zooniverse_Classes[i][line["c_snid_1"]] += line["#1"]
    ResultsTable_Zooniverse_Classes[i][line["c_snid_2"]] += line["#2"]
    ResultsTable_Zooniverse_Classes[i][line["c_snid_3"]] += line["#3"]
    ResultsTable_Zooniverse_Classes[i][line["c_snid_4"]] += line["#4"]
    ResultsTable_Zooniverse_Classes[i][line["c_snid_5"]] += line["#5"]
    top_line = [line["c_snid_1"], line["c_snid_2"], line["c_snid_3"],line["c_snid_4"], line["c_snid_5"]]
    classes = np.unique(top_line)
    votes = ResultsTable_Zooniverse_Classes[i][classes.tolist()]
    vote_data = []
    for j in votes:
        vote_data.append(j)
    best = np.where(np.asarray(top_line) == classes[np.argmax(vote_data)])[0][0]
    ResultsTable_Zooniverse_Classes[i]["rlap"] = line["rlap_{}".format(best + 1)]
    ResultsTable_Zooniverse_Classes[i]["z"] = line["z_snid_{}".format(best + 1)]

In [29]:
len(joined_results)

8763

In [30]:
joined_results

Version,ZTF_Name,z_sntemplate,z_rlap,z_snid,z_snid_err,z_level,rank_1,sntemplate_1,rlap_1,c_snid_1,z_snid_1,z_snid_err_1,age_1,age_flag_1,rank_2,sntemplate_2,rlap_2,c_snid_2,z_snid_2,z_snid_err_2,age_2,age_flag_2,rank_3,sntemplate_3,rlap_3,c_snid_3,z_snid_3,z_snid_err_3,age_3,age_flag_3,rank_4,sntemplate_4,rlap_4,c_snid_4,z_snid_4,z_snid_err_4,age_4,age_flag_4,rank_5,sntemplate_5,rlap_5,c_snid_5,z_snid_5,z_snid_err_5,age_5,age_flag_5,subject_ids,NoMatch,#1,#2,#3,#4,#5
str64,str64,str12,float64,float64,float64,int64,int64,str12,float64,str7,float64,float64,float64,int64,int64,str12,float64,str7,float64,float64,float64,int64,int64,str12,float64,str7,float64,float64,float64,int64,int64,str12,float64,str7,float64,float64,float64,int64,int64,str12,float64,str7,float64,float64,float64,int64,int32,int32,int32,int32,int32,int32,int32
AT2018hrg_20181103_P60_v1.ascii,AT2018hrg,sn2006bp,5.84,0.0068,0.0091,1,1,sn2006bp,5.84,II-norm,0.0068,0.0091,4.2,0,3,sn1987A,5.76,II-norm,0.0085,0.0094,-78.5,0,6,sn06bp,5.71,II-norm,0.0064,0.0092,7.2,0,18,sn1996cb,5.34,IIb,0.0158,0.0101,-18.0,0,35,lbv01ac,5.06,LBV,-0.0005,0.0045,-99.9,2,53623564,5,0,0,0,1,4
SN2017iuk_20181204_Keck1_v1.ascii,SN2017iuk,kcSB5,5.66,0.0378,0.0028,1,1,kcSB5,5.66,Gal,0.0378,0.0028,-99.9,2,2,kcSB1,5.43,Gal,0.037000000000000005,0.0028,-99.9,2,3,kcSB2,5.11,Gal,0.0369,0.003,-99.9,2,7,sn98bw,4.35,Ic-BL,0.0094,0.0179,15.7,0,11,sn2005la,3.83,Ibn,0.0197,0.0033,14.0,0,53623565,2,1,1,6,0,0
ZTF17aaajnki_20180320_P60_v1.ascii,ZTF17aaajnki,kcSB2,17.99,-0.0,0.0015,1,1,kcSB2,17.99,Gal,-0.0,0.0015,-99.9,2,2,kcSB1,17.3,Gal,0.0001,0.0016,-99.9,2,3,PS1-10bzj,10.61,Ic-SLSN,0.0003,0.0026,6.55,0,5,kcSB4,5.8,Gal,-0.0002,0.0041,-99.9,2,8,sn07it,5.28,II-norm,0.0007,0.0051,191.0,0,53623566,1,7,2,0,0,0
ZTF17aaajnki_20180320_P60_v2.ascii,ZTF17aaajnki,kcSB2,18.27,-0.0004,0.0015,1,1,kcSB2,18.27,Gal,-0.0004,0.0015,-99.9,2,2,kcSB1,17.68,Gal,-0.0003,0.0015,-99.9,2,3,PS1-10bzj,11.43,Ic-SLSN,-0.0001,0.0024,6.55,0,5,kcSB4,5.82,Gal,-0.0006,0.004,-99.9,2,8,sn07it,5.14,II-norm,0.0003,0.0051,191.0,0,53623567,1,10,2,0,0,0
ZTF17aaaobyl_20180826_P60_v2.ascii,ZTF17aaaobyl,agn,7.95,0.0017,0.003,1,1,agn,7.95,AGN,0.0017,0.003,-99.9,2,2,kcSB5,7.77,Gal,0.0021,0.0025,-99.9,2,3,kcSB4,7.62,Gal,0.0017,0.0025,-99.9,2,4,kcSc,7.52,Gal,0.0022,0.0025,-99.9,2,5,sn98S,7.15,IIn,0.004,0.0057,422.5,0,53623568,2,1,2,3,1,0
ZTF17aaapufz_20190304_P200_v1.ascii,ZTF17aaapufz,kcSc,28.52,0.0025,0.0006,1,1,kcSc,28.52,Gal,0.0025,0.0006,-99.9,2,2,kcSB4,26.83,Gal,0.0022,0.0007,-99.9,2,3,kcSB5,22.66,Gal,0.0025,0.0008,-99.9,2,8,agn,14.22,AGN,0.0023,0.0014,-99.9,2,9,qso,12.11,QSO,0.0025,0.0016,-100.0,2,53623569,1,9,0,1,0,0
ZTF17aaaukqn_20180904_P60_v1.ascii,ZTF17aaaukqn,sn97ef,6.95,0.3262,0.0146,0,1,sn97ef,6.95,Ic-BL,0.3262,0.0146,-14.0,0,7,sn2003W,4.54,Ia-norm,0.4743,0.0108,28.0,0,10,sn2005cf,4.3,Ia-norm,0.4432,0.0134,-6.6,0,16,sn1999gp,4.12,Ia-91T,0.4702,0.0123,35.1,0,17,sn03du,4.11,Ia-norm,0.4409,0.0122,-5.0,0,53623570,4,7,0,0,0,0
ZTF17aaawgkc_20180822_P60_v1.ascii,ZTF17aaawgkc,sn1995F,5.91,0.2735,0.0108,0,1,sn1995F,5.91,Ic-norm,0.2735,0.0108,88.8,1,3,sn93J,5.88,IIb,0.2642,0.0078,102.0,0,7,sn2008aq,5.64,IIb,0.2654,0.0075,101.0,1,12,sn2005az,5.42,Ic-norm,0.27,0.0098,87.3,0,16,sn2008D,5.24,Ib-norm,0.2808,0.0133,63.9,0,53623571,10,0,0,0,0,1
ZTF17aaayjzy_20191214_P60_v1.ascii,ZTF17aaayjzy,kcSB4,20.67,0.0479,0.0012,1,1,kcSB4,20.67,Gal,0.0479,0.0012,-99.9,2,2,kcSB5,19.9,Gal,0.0482,0.0012,-99.9,2,3,kcSc,18.67,Gal,0.0482,0.0013,-99.9,2,6,agn,13.59,AGN,0.0479,0.0021,-99.9,2,9,qso,10.66,QSO,0.048,0.0024,-100.0,2,53623572,1,4,2,1,1,0


In [31]:
ResultsTable_Zooniverse_Classes.to_pandas().to_csv("ClassTotalsAll.csv")

In [32]:
final_rcf_table = Table.from_pandas(pd.read_hdf(r"C:\Users\20xha\Documents\Caltech\Research/final_rcf_table.h5"))

In [33]:
sample = Table.read("C:/Users/20xha/Documents/Caltech/Research/superfit/ML_sample.ascii", format = "ascii")
sample.rename_column('col1', 'ZTF_Name')
sample.rename_column('col2', "Class")
sample.rename_column('col3', "redshift")
sample.rename_column('col8', "Version")

In [34]:
Joined_ZTF = astropy.table.join(ResultsTable_Zooniverse_Classes, sample)

In [35]:
types = ['Ia', 'Ia-norm', 'Ia-91T', 'Ia-91bg', 'Ia-csm', 'Ia-02cx', 'Ia-03fg',
                           'Ib', 'Ib-norm', 'IIb', 'Ibn', 'Ic', 'Ic-norm', 'Ic-BL', 'Ic-SLSN', 'II', 'II-norm', 'IIn',
                           'NotSN', 'AGN', 'Gal', 'LBV', 'M-star', 'QSO', 'C-star', 'TDE', "NoMatch"]

In [36]:
cutoffs = [6,7,8,9,10,11]
rlap_cutoffs = [0, 5, 10, 15, 20]

In [37]:
Joined_ZTF[np.where(Joined_ZTF["Version"] == "ZTF18aaegvyd_20190625_P60_v1.ascii")]

ZTF_Name,Version,subject_ids,Ia,Ia-norm,Ia-91T,Ia-91bg,Ia-csm,Ia-02cx,Ia-03fg,Ib,Ib-norm,IIb,Ibn,Ic,Ic-norm,Ic-BL,Ic-SLSN,II,II-norm,IIn,NotSN,AGN,Gal,LBV,M-star,QSO,C-star,TDE,NoMatch,rlap,z,Class,redshift,col4,col5,col6,col7,col9
str64,str64,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,float32,float32,str12,float64,float64,float64,float64,float64,int32
ZTF18aaegvyd,ZTF18aaegvyd_20190625_P60_v1.ascii,53623655,0,1,0,0,0,0,0,0,1,0,0,0,3,0,0,0,3,0,0,0,0,0,0,0,0,0,0,5.18,0.0281,II,0.0235,nan,2458659.5,2458638.66,20.84,0


In [38]:
Classes_Final = Table(
                names=("ZTF_Name", "Version", "Zooniverse_c_broad", "agreement_broad", "Zooniverse_c", "rlap", "agreement"
                ),
                meta={"name": "Spectrum Results after Zooniverse"},
                dtype=("U64", "U64", "U64", "int32", "U64", "float32", "int32"
                      )
                )
for line in Joined_ZTF:
    row = []
    row.append(line["ZTF_Name"])
    row.append(line["Version"])
    best = []
    best_num = []
    broad_types = ["Ia", "II", "Ib", "Ic"]
    broad_type_total = [0,0,0,0]
    for i in types:
        if(line[i] >= 0):
            best.append(i)
            votes = line[i]
            best_num.append(votes)
    for i in range(len(best)):
        for j in range(len(broad_types)):
            if(broad_types[j] in best[i]):
                broad_type_total[j] += best_num[i]
    if(np.max(broad_type_total) < np.max(best_num)):
        row.append(best[np.argmax(best_num)])
        row.append(np.max(best_num))
    else:
        row.append(broad_types[np.argmax(broad_type_total)])
        row.append(np.max(broad_type_total))
    if(len(best) != 0):
        row.append(best[np.argmax(best_num)])
        row.append(line["rlap"])
        row.append(np.max(best_num))
        Classes_Final.add_row(row)

In [39]:
superfit_results = Table.from_pandas(pd.read_csv(r"C:\Users\20xha\Documents\Caltech\Research\superfit\superfit\superfit_classes.csv"))

In [40]:
len(np.unique(Joined_ZTF["ZTF_Name"]))

4400

In [41]:
combined = astropy.table.join(superfit_results, Classes_Final, keys = ["Version"])
outer = astropy.table.join(superfit_results, Classes_Final, keys = ["Version"], join_type = "outer").to_pandas()

C:\Users\20xha\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3418: TableReplaceWarning: converted column 'agreement_broad' from integer to float
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\20xha\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3418: TableReplaceWarning: converted column 'agreement' from integer to float
  exec(code_obj, self.user_global_ns, self.user_ns)


In [42]:
missing = Table.from_pandas(outer[outer['Superfit_c'].isna()])

In [43]:
missing

Version,Resolution,Superfit_c,Superfit_z,ZTF_Name,Zooniverse_c_broad,agreement_broad,Zooniverse_c,rlap,agreement
float64,float64,float64,float64,float64,float64,float64,float64,float32,float64


In [44]:
missing.to_pandas().to_csv("missing.csv", index = False)

In [45]:
print(len(np.unique(Classes_Final["Version"])))
print(len(np.unique(superfit_results["Version"])))

8757
8813


In [46]:
len(np.unique(superfit_results["Version"]))

8813

In [47]:
names = np.unique(combined["ZTF_Name"])

In [48]:
print(len(names))
print(len(np.unique(Classes_Final["ZTF_Name"])))

4400
4400


In [51]:
counter = 0
for name in names:
    loc = np.where(combined["ZTF_Name"] == name)
    isII = False
    for j in combined[loc]:
        if(j["agreement_broad"] >= 6):
            superfitII = "II" in j["Superfit_c"]
            zooniverseII = "II" in j["Zooniverse_c_broad"]
            if(superfitII and zooniverseII):
                isII = True
    #if(isII):
        #print(combined[loc]["ZTF_Name", "Superfit_c", "Zooniverse_c"])
    counter += isII

In [52]:
counter

589

In [104]:
tables = []
for cut in cutoffs:
    Classes = Table(
                    names=("ZTF_Name", "Version", "Real_c", "Zooniverse_c"
                    ),
                    meta={"name": "Spectrum Results after Zooniverse"},
                    dtype=("U64", "U64", "U64", "U64",
                          )
                    )
    for line in Joined_ZTF:
        row = []
        row.append(line["ZTF_Name"])
        row.append(line["Version"])
        row.append(line["sn_type"])
        best = []
        best_num = []
        for i in types:
            if(line[i] >= cut):
                best.append(i)
                best_num.append(line[i])
        if(len(best) != 0):
            row.append(best[np.argmax(best_num)])
            Classes.add_row(row)
    tables.append([cut, Classes])

KeyError: 'sn_type'

In [ ]:
tables = []
for cut in rlap_cutoffs:
    Classes = Table(
                    names=("ZTF_Name", "Version", "Real_c", "Zooniverse_c"
                    ),
                    meta={"name": "Spectrum Results after Zooniverse"},
                    dtype=("U64", "U64", "U64", "U64",
                          )
                    )
    for line in Joined_ZTF_2018:
        row = []
        row.append(line["ZTF_Name"])
        row.append(line["Version"])
        row.append(line["sn_type"])
        best = []
        best_num = []
        for i in types:
            if(line[i] >= 9 and line["z"] < .2):
                if(line["rlap"] >= cut):
                    best.append(i)
                    best_num.append(line[i])
        if(len(best) != 0):
            row.append(best[np.argmax(best_num)])
            Classes.add_row(row)
    tables.append([cut, Classes])

In [ ]:
negativeII = 0
positiveII = 0
for j in Joined_ZTF_2018:
    correct_1a = "II" in j["sn_type"]
    if(correct_1a):
        positiveII += 1
    if(not(correct_1a)):
        negativeII += 1

In [ ]:
cutoff_confidence = [] 
for i in tables:
    truepositive = 0
    falsepositive = 0
    for j in i[1]:
        correct_1a = "II" in j["Real_c"]
        classified_1a = "II" in j["Zooniverse_c"]
        if(correct_1a == True and classified_1a == True):
            truepositive += 1
        if(correct_1a != True and classified_1a == True):
            falsepositive += 1
    cutoff_confidence.append([i[0],truepositive/(positiveII),falsepositive/(negativeII),len(i[1]), truepositive, falsepositive])
cutoff_confidence = np.asarray(cutoff_confidence)

In [ ]:
cutoff_confidence

In [ ]:
fig = plt.figure(figsize = (10,10))
plt.scatter(cutoff_confidence[:,2], cutoff_confidence[:,1], color = "blue")
for i in cutoff_confidence:
    plt.text(i[2]+.00002,i[1]+.002,"$>$"+str(i[0]))
plt.title("TPR vs. FPR for different # in agreement II")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")

In [ ]:
negativeI = 0
positiveI = 0
for j in Joined_ZTF_2018:
    correct_1a = "Ia" in j["sn_type"]
    if(correct_1a):
        positiveI += 1
    if(not(correct_1a)):
        negativeI += 1

In [ ]:
cutoff_confidence_Ia = [] 
for i in tables:
    truepositive = 0
    falsepositive = 0
    for j in i[1]:
        correct_1a = "Ia" in j["Real_c"]
        classified_1a = "Ia" in j["Zooniverse_c"]
        if(correct_1a == True and classified_1a == True):
            truepositive += 1
        if(correct_1a != True and classified_1a == True):
            falsepositive += 1
    cutoff_confidence_Ia.append([i[0],truepositive/(positiveI),falsepositive/(negativeI),len(i[1]), truepositive, falsepositive])
cutoff_confidence_Ia = np.asarray(cutoff_confidence_Ia)

In [ ]:
fig = plt.figure(figsize = (10,10))
plt.scatter(cutoff_confidence_Ia[:,2], cutoff_confidence_Ia[:,1], color = "blue")
for i in cutoff_confidence_Ia:
    plt.text(i[2]+.00002,i[1]+.002,"$>$"+str(i[0]))
plt.title("TPR vs. FPR for different # in agreement Ia")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")

In [ ]:
positiveI

In [ ]:
positiveII

In [ ]:
cutoff_confidence_Ia = [] 
for i in tables:
    truepositive = 0
    falsepositive = 0
    for j in i[1]:
        correct_1a = "Ia" in j["Real_c"]
        classified_1a = "Ia" in j["Zooniverse_c"]
        if(correct_1a == True and classified_1a == True):
            truepositive += 1
        if(correct_1a != True and classified_1a == True):
            print(j)
    break

In [ ]:
tables[-1][1]

In [ ]:
np.sum(ResultsTable_Zooniverse_Classes["Ia-norm"] > 6)

In [ ]:
positiveI

In [ ]:
negativeIbIc = 0
positiveIbIc = 0
index = 0
indexes = []
for j in Joined_ZTF_2018:
    correct_1a = ("Ib" in j["sn_type"]) or ("Ic" in j["sn_type"]) 
    if(correct_1a):
        positiveIbIc += 1
        indexes.append(index)
    if(not(correct_1a)):
        negativeIbIc += 1
    index += 1

In [ ]:
cutoff_confidence_IbIc = [] 
for i in tables:
    truepositiveIbIc = 0
    falsepositiveIbIc = 0
    JoinedResults = i[1]
    for j in JoinedResults:
        correct_1a = ("Ib" in j["Real_c"])
        classified_1a = ("Ib" in j["Zooniverse_c"])
        if(correct_1a == True and classified_1a == True):
            truepositiveIbIc += 1
        if(correct_1a != True and classified_1a == True):
            falsepositiveIbIc += 1
            
        correct_1a = ("Ic" in j["Real_c"])
        classified_1a = ("Ic" in j["Zooniverse_c"])
        if(correct_1a == True and classified_1a == True):
            truepositiveIbIc += 1
        if(correct_1a != True and classified_1a == True):
            falsepositiveIbIc += 1
        
    cutoff_confidence_IbIc.append([i[0],truepositiveIbIc/(positiveIbIc),falsepositiveIbIc/(negativeIbIc),len(JoinedResults), truepositiveIbIc, falsepositiveIbIc])
cutoff_confidence_IbIc = np.asarray(cutoff_confidence_IbIc)

In [ ]:
fig = plt.figure(figsize = (10,10))
plt.scatter(cutoff_confidence_IbIc[:,2], cutoff_confidence_IbIc[:,1], color = "blue")
for i in cutoff_confidence_IbIc:
    plt.text(i[2]+.00002,i[1]+.002,"$>$"+str(i[0]))
plt.title("TPR vs. FPR for different # in agreement (Striped Envolope Ib or Ic)")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")

In [ ]:
for i in tables[-2][1]:
    if("Ib" in i["Real_c"]):
        print(i["Real_c"], i["Zooniverse_c"])

In [ ]:
for i in tables[-2][1]:
    if("Ib" in i["Zooniverse_c"]):
        print(i["Real_c"], i["Zooniverse_c"])

In [ ]:
tables[0][1][tables[0][1]["Zooniverse_c"]=="NoMatch"]

In [ ]:
joined_results

Further Analysis
Weighing Users Plot number of classifications vs accuracy
Weighting Schemes
Isolate Users or subclasses
Cluster things that are wrong
PanStarrs Subtraction
Age vs Accuracy
Phase of the spectra vs Accuracyclassification[ccs_workflow]

In [ ]:
wanted = classification[ccs_workflow]

In [ ]:
users = np.unique(wanted["user_name"])

In [ ]:
len(users)

In [ ]:
location_list = []
counter = 0
for i in users:
    counter += 1
    locations = np.where(wanted["user_name"] == i)
    location_list.append(np.asarray([i,locations[0],len(locations[0])]))
    if(counter % 100 == 0):
        print(counter)
location_list = np.asarray(location_list)

In [ ]:
plt.hist(location_list[:, 2], bins = 100)

In [ ]:
super_users = location_list[np.where(location_list[:,2] > 250)]

In [ ]:
np.unique(location_list[:,2])

In [ ]:
results = []
for amount_done in np.unique(location_list[:,2]):
    right = 0
    total = 0
    for line in location_list[np.where(location_list[:,2] == amount_done)]:
        these_classes = classification[ccs_workflow][line[1]]
        for i in these_classes:
            subject_data = json.loads(i["subject_data"])
            annotations = json.loads(i["annotations"])
            value = annotations[0]["value"]
            ztf_name = subject_data[str(i["subject_ids"])]["!ZTF_Name"]
            ztf_version = subject_data[str(i["subject_ids"])]["!ZTF_Version"]
            type_location = np.where(final_rcf_table["ZTF_Name"] == ztf_name)[0]
            if(len(type_location) > 0 and value != None):
                type_location = type_location[0]
                snid_location = np.where(snidoutput["Version"] == ztf_version+".ascii")[0][0]
                response_number = response_dict[value]
                if(response_number != 0):
                    snid_class = snidoutput[snid_location]["c_snid_{}".format(response_number)]
                    actual_class = final_rcf_table[type_location]["sn_type"]
                    if(actual_class == snid_class):
                        right += 1
                    elif(actual_class in snid_class):
                        right += 1
                    elif("Ia" in actual_class and "Ia" in snid_class):
                        right += 1
                    elif("II" in actual_class and "II" in snid_class):
                        right += 1
                    elif("Ib" in actual_class and "Ib" in snid_class):
                        right += 1
                    elif("Ic" in actual_class and "Ic" in snid_class):
                        right += 1
                    total += 1
    results.append([amount_done, right, total, right/total])

In [ ]:
results = np.asarray(results)

In [ ]:
plt.scatter(results[:,0], results[:,3])

In [ ]:
snidoutput[0]["c_snid_{}".format(1)]

In [ ]:
np.where(final_rcf_table["ZTF_Name"] == "ZTF18aabssth")[0]

In [ ]:
np.unique(final_rcf_table["sn_type"])

In [ ]:
np.unique(snidoutput["c_snid_1"])

In [ ]:
ztf_version+"ascii"

In [ ]:
tables_age = []
for cut in rlap_cutoffs:
    Classes = Table(
                    names=("ZTF_Name", "Version", "Real_c", "Zooniverse_c"
                    ),
                    meta={"name": "Spectrum Results after Zooniverse"},
                    dtype=("U64", "U64", "U64", "U64",
                          )
                    )
    for line in Joined_ZTF_2018:
        row = []
        row.append(line["ZTF_Name"])
        row.append(line["Version"])
        row.append(line["sn_type"])
        best = []
        best_num = []
        for i in types:
            if(line[i] >= 9):
                if(line["rlap"] >= cut):
                    best.append(i)
                    best_num.append(line[i])
        if(len(best) != 0):
            row.append(best[np.argmax(best_num)])
            Classes.add_row(row)
    tables_age.append([cut, Classes])

In [ ]:
snid_superfit_results = Table.read(r"C:\Users\20xha\Documents\GitHub\supernova-spectrum-analysis\superfit/results_2018_all_exact_z.txt", format = "ascii")
snid_superfit_results.rename_column('name', 'Version')

In [ ]:

tprfpr_snid = Table.read(r"C:\Users\20xha\Documents\GitHub\supernova-spectrum-analysis\superfit/tprfpr_snid.ascii", format = "ascii")

In [ ]:
snid_superfit_results[0]

In [ ]:
joined_phase = astropy.table.join(Classes_Final, snid_superfit_results)

In [ ]:
phase_cutoffs = np.linspace(np.min(joined_phase["phase"]), np.max(joined_phase["phase"]), 100)

In [ ]:
joined_phase[0]

In [ ]:
negativeIa = 0
positiveIa = 0
for j in joined_phase:
    correct_1a = "Ia" in j["classification"]
    if(correct_1a):
        positiveIa += 1
    if(not(correct_1a)):
        negativeIa += 1

In [ ]:
joined_phase

In [ ]:
tprfpr_zooniverse = Table(
                names=("phase", "agreement", "tpr_zooniverse", "fpr_zooniverse"
                ),
                meta={"name": "Comparison of Three Programs"},
                dtype=("float64", "float64", "float64", "float64"
                      )) 
for phase in phase_cutoffs:
    for agreement in cutoffs:
        truepositive = [0, 0, 0]
        falsepositive = [0, 0, 0]
        for i in joined_phase:
            if(i["phase"] > phase and i["agreement"] >= agreement):
                c_actual = "Ia" in i["Real_c"]

                c_snid = "Ia" in i["Zooniverse_c"]
                if(c_actual == True and c_snid == True):
                    truepositive[0] += 1
                if(c_actual != True and c_snid == True):
                    falsepositive[0] += 1
        row = [phase, agreement, truepositive[0]/positiveIa, falsepositive[0]/negativeIa]
        tprfpr_zooniverse.add_row(row)   

In [ ]:
positiveIa

In [ ]:
62/441

In [ ]:
fig = plt.figure(figsize = (10,10))
plt.scatter(tprfpr_zooniverse["fpr_zooniverse"], tprfpr_zooniverse["tpr_zooniverse"], color = "orange")
plt.plot(tprfpr_snid["fpr_snid"], tprfpr_snid["tpr_snid"])
plt.title("TPR vs. FPR for different # in agreement Ia")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")

In [ ]:
tprfpr_zooniverse_superfit_Ia = Table(
                names=("phase", "agreement", "tpr_zooniverse", "fpr_zooniverse"
                ),
                meta={"name": "Comparison of Three Programs"},
                dtype=("float64", "float64", "float64", "float64"
                      ))
for phase in phase_cutoffs:
    for agreement in cutoffs:
        truepositive = [0, 0, 0]
        falsepositive = [0, 0, 0]
        for i in joined_phase:
            if(i["phase"] > phase and i["agreement"] >= agreement):
                c_actual = "Ia" in i["Real_c"]
                
                c_zoonivserse = "Ia" in i["Zooniverse_c"]
                c_superfit = "Ia" in i["SF_fit_1"]
                c = c_zoonivserse and c_superfit
                if(c_actual == True and c == True):
                    truepositive[0] += 1
                if(c_actual != True and c == True):
                    falsepositive[0] += 1
        row = [phase, agreement, truepositive[0]/positiveIa, falsepositive[0]/negativeIa]
        tprfpr_zooniverse_superfit_Ia.add_row(row)

fig = plt.figure(figsize = (10,10))
plt.scatter(tprfpr_zooniverse_superfit_Ia["fpr_zooniverse"], tprfpr_zooniverse_superfit_Ia["tpr_zooniverse"], color = "orange", label = "Superfit and Zooniverse")
plt.plot(tprfpr_snid["fpr_snid"], tprfpr_snid["tpr_snid"], label = "SNID")
plt.title("TPR vs. FPR for different # in agreement Ia (Superfit and Zooniverse)")
plt.legend(loc="upper left")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")

In [ ]:
negativeII = 0
positiveII = 0
for j in joined_phase:
    correct_1a = "II" in j["classification"]
    if(correct_1a):
        positiveII += 1
    if(not(correct_1a)):
        negativeII += 1

In [ ]:
tprfpr_snid_II = Table(
                names=("rlap", "tpr_snid", "fpr_snid"
                ),
                meta={"name": "Comparison of Three Programs"},
                dtype=("float64", "float64", "float64"
                      )) 
for rlap in np.linspace(0, 25, 251):
    truepositive = [0, 0, 0]
    falsepositive = [0, 0, 0]
    for i in joined_phase:
        if(i["rlap"] >= rlap):
            c_actual = "II" in i["Real_c"]

            c_snid = "II" in i["c_snid"]
            if(c_actual == True and c_snid == True):
                truepositive[0] += 1
            if(c_actual != True and c_snid == True):
                falsepositive[0] += 1
    row = [rlap, truepositive[0]/positiveII, falsepositive[0]/negativeII]
    tprfpr_snid_II.add_row(row)   

In [ ]:
tprfpr_zooniverse_II = Table(
                names=("phase", "agreement", "tpr_zooniverse", "fpr_zooniverse"
                ),
                meta={"name": "Comparison of Three Programs"},
                dtype=("float64", "float64", "float64", "float64"
                      )) 
for phase in phase_cutoffs:
    for agreement in cutoffs:
        truepositive = [0, 0, 0]
        falsepositive = [0, 0, 0]
        for i in joined_phase:
            if(i["phase"] > phase and i["agreement"] >= agreement):
                c_actual = "II" in i["Real_c"]

                c_snid = "II" in i["Zooniverse_c"]
                if(c_actual == True and c_snid == True):
                    truepositive[0] += 1
                if(c_actual != True and c_snid == True):
                    falsepositive[0] += 1
        row = [phase, agreement, truepositive[0]/positiveII, falsepositive[0]/negativeII]
        tprfpr_zooniverse_II.add_row(row)   

In [ ]:
fig = plt.figure(figsize = (10,10))
plt.scatter(tprfpr_zooniverse_II["fpr_zooniverse"], tprfpr_zooniverse_II["tpr_zooniverse"], color = "orange")
plt.plot(tprfpr_snid_II["fpr_snid"], tprfpr_snid_II["tpr_snid"])
plt.title("TPR vs. FPR for different # in agreement II")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")

In [ ]:
import matplotlib

In [ ]:
plt.rcParams.update({'font.size': 20})

In [ ]:
tprfpr_zooniverse_superfit_II = Table(
                names=("phase", "agreement", "tpr_zooniverse", "fpr_zooniverse"
                ),
                meta={"name": "Comparison of Three Programs"},
                dtype=("float64", "float64", "float64", "float64"
                      ))
for phase in phase_cutoffs:
    for agreement in cutoffs:
        truepositive = [0, 0, 0]
        falsepositive = [0, 0, 0]
        for i in joined_phase:
            if(i["phase"] > phase and i["agreement"] >= agreement):
                c_actual = "II" in i["Real_c"]
                
                c_zoonivserse = "II" in i["Zooniverse_c"]
                c_superfit = "II" in i["SF_fit_1"]
                c = c_zoonivserse and c_superfit
                if(c_actual == True and c == True):
                    truepositive[0] += 1
                if(c_actual != True and c == True):
                    falsepositive[0] += 1
        row = [phase, agreement, truepositive[0]/positiveII, falsepositive[0]/negativeII]
        tprfpr_zooniverse_superfit_II.add_row(row)

fig = plt.figure(figsize = (10,10))
plt.scatter(tprfpr_zooniverse_superfit_II[1:]["fpr_zooniverse"], tprfpr_zooniverse_superfit_II[1:]["tpr_zooniverse"], color = "orange", label = "Superfit and Zooniverse")
plt.plot(tprfpr_snid_II["fpr_snid"], tprfpr_snid_II["tpr_snid"], label = "SNID")
plt.scatter(0, tprfpr_zooniverse_superfit_II["tpr_zooniverse"][np.argmax(tprfpr_zooniverse_superfit_II["tpr_zooniverse"])], s=640, marker=(5, 1), color = "green")
#plt.title("TPR vs. FPR for different # in agreement (Type II) (Superfit and Zooniverse)")
plt.legend(loc="upper left")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")

In [ ]:
tprfpr_zooniverse_superfit_II[np.argmax(tprfpr_zooniverse_superfit_II["tpr_zooniverse"])]

In [ ]:
tprfpr_zooniverse_superfit_II = Table(
                names=("phase", "agreement", "tpr_zooniverse", "fpr_zooniverse"
                ),
                meta={"name": "Comparison of Three Programs"},
                dtype=("float64", "float64", "float64", "float64"
                      ))
for phase in phase_cutoffs:
    for agreement in cutoffs:
        truepositive = [0, 0, 0]
        falsepositive = [0, 0, 0]
        for i in joined_phase:
            if(i["phase"] > phase and i["agreement"] >= agreement):
                c_actual = "II" == i["Real_c"]
                
                c_zoonivserse = "II-norm" == i["Zooniverse_c"]
                c_superfit = "II" == i["SF_fit_1"]
                c = c_zoonivserse and c_superfit
                if(c_actual == True and c == True):
                    truepositive[0] += 1
                if(c_actual != True and c == True):
                    falsepositive[0] += 1
        row = [phase, agreement, truepositive[0]/positiveII, falsepositive[0]/negativeII]
        tprfpr_zooniverse_superfit_II.add_row(row)

fig = plt.figure(figsize = (10,10))
plt.scatter(tprfpr_zooniverse_superfit_II["fpr_zooniverse"], tprfpr_zooniverse_superfit_II["tpr_zooniverse"], color = "orange")
plt.plot(tprfpr_snid_II["fpr_snid"], tprfpr_snid_II["tpr_snid"])
plt.title("TPR vs. FPR for different # in agreement II-norm (Superfit and Zooniverse)")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")

In [ ]:
negativeIb = 0
positiveIb = 0
for j in joined_phase:
    correct_1a = "Ib" in j["classification"]
    if(correct_1a):
        positiveIb += 1
    if(not(correct_1a)):
        negativeIb += 1

In [ ]:
tprfpr_snid_Ib = Table(
                names=("rlap", "tpr_snid", "fpr_snid"
                ),
                meta={"name": "Comparison of Three Programs"},
                dtype=("float64", "float64", "float64"
                      )) 
for rlap in np.linspace(0, 25, 251):
    truepositive = [0, 0, 0]
    falsepositive = [0, 0, 0]
    for i in joined_phase:
        if(i["rlap"] >= rlap):
            c_actual = "Ib" in i["Real_c"]

            c_snid = "Ib" in i["c_snid"]
            if(c_actual == True and c_snid == True):
                truepositive[0] += 1
            if(c_actual != True and c_snid == True):
                falsepositive[0] += 1
    row = [rlap, truepositive[0]/positiveIb, falsepositive[0]/negativeIb]
    tprfpr_snid_Ib.add_row(row)

In [ ]:
tprfpr_zooniverse_Ib = Table(
                names=("phase", "agreement", "tpr_zooniverse", "fpr_zooniverse"
                ),
                meta={"name": "Comparison of Three Programs"},
                dtype=("float64", "float64", "float64", "float64"
                      )) 
for phase in phase_cutoffs:
    for agreement in cutoffs:
        truepositive = [0, 0, 0]
        falsepositive = [0, 0, 0]
        for i in joined_phase:
            if(i["phase"] > phase and i["agreement"] >= agreement):
                c_actual = "Ib" in i["Real_c"]

                c_snid = "Ib" in i["Zooniverse_c"]
                if(c_actual == True and c_snid == True):
                    truepositive[0] += 1
                if(c_actual != True and c_snid == True):
                    falsepositive[0] += 1
        row = [phase, agreement, truepositive[0]/positiveIb, falsepositive[0]/negativeIb]
        tprfpr_zooniverse_Ib.add_row(row)   

In [ ]:
fig = plt.figure(figsize = (10,10))
plt.scatter(tprfpr_zooniverse_Ib["fpr_zooniverse"], tprfpr_zooniverse_Ib["tpr_zooniverse"], color = "orange")
plt.plot(tprfpr_snid_Ib["fpr_snid"], tprfpr_snid_Ib["tpr_snid"])
plt.title("TPR vs. FPR for different # in agreement Ib")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")

In [ ]:
tprfpr_zooniverse_superfit_Ib = Table(
                names=("phase", "agreement", "tpr_zooniverse", "fpr_zooniverse"
                ),
                meta={"name": "Comparison of Three Programs"},
                dtype=("float64", "float64", "float64", "float64"
                      ))
for phase in phase_cutoffs:
    for agreement in cutoffs:
        truepositive = [0, 0, 0]
        falsepositive = [0, 0, 0]
        for i in joined_phase:
            if(i["phase"] > phase and i["agreement"] >= agreement):
                c_actual = "Ib" in i["Real_c"]
                
                c_zoonivserse = "Ib" in i["Zooniverse_c"]
                c_superfit = "Ib" in i["SF_fit_1"]
                c = c_zoonivserse and c_superfit
                if(c_actual == True and c == True):
                    truepositive[0] += 1
                if(c_actual != True and c == True):
                    falsepositive[0] += 1
        row = [phase, agreement, truepositive[0]/positiveII, falsepositive[0]/negativeII]
        tprfpr_zooniverse_superfit_Ib.add_row(row)

fig = plt.figure(figsize = (10,10))
plt.scatter(tprfpr_zooniverse_superfit_Ib["fpr_zooniverse"], tprfpr_zooniverse_superfit_Ib["tpr_zooniverse"], color = "orange")
plt.plot(tprfpr_snid_Ib["fpr_snid"], tprfpr_snid_Ib["tpr_snid"])
plt.title("TPR vs. FPR for different # in agreement Ib (Superfit and Zooniverse)")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")

In [ ]:
negativeIc = 0
positiveIc = 0
for j in joined_phase:
    correct_1a = "Ic" in j["classification"]
    if(correct_1a):
        positiveIc += 1
    if(not(correct_1a)):
        negativeIc += 1

In [ ]:
tprfpr_snid_Ic = Table(
                names=("rlap", "tpr_snid", "fpr_snid"
                ),
                meta={"name": "Comparison of Three Programs"},
                dtype=("float64", "float64", "float64"
                      )) 
for rlap in np.linspace(0, 25, 251):
    truepositive = [0, 0, 0]
    falsepositive = [0, 0, 0]
    for i in joined_phase:
        if(i["rlap"] >= rlap):
            c_actual = "Ic" in i["Real_c"]

            c_snid = "Ic" in i["c_snid"]
            if(c_actual == True and c_snid == True):
                truepositive[0] += 1
            if(c_actual != True and c_snid == True):
                falsepositive[0] += 1
    row = [rlap, truepositive[0]/positiveIc, falsepositive[0]/negativeIc]
    tprfpr_snid_Ic.add_row(row)

In [ ]:
tprfpr_zooniverse_Ic = Table(
                names=("phase", "agreement", "tpr_zooniverse", "fpr_zooniverse"
                ),
                meta={"name": "Comparison of Three Programs"},
                dtype=("float64", "float64", "float64", "float64"
                      )) 
for phase in phase_cutoffs:
    for agreement in cutoffs:
        truepositive = [0, 0, 0]
        falsepositive = [0, 0, 0]
        for i in joined_phase:
            if(i["phase"] > phase and i["agreement"] >= agreement):
                c_actual = "Ic" in i["Real_c"]

                c_snid = "Ic" in i["Zooniverse_c"]
                if(c_actual == True and c_snid == True):
                    truepositive[0] += 1
                if(c_actual != True and c_snid == True):
                    falsepositive[0] += 1
        row = [phase, agreement, truepositive[0]/positiveIc, falsepositive[0]/negativeIc]
        tprfpr_zooniverse_Ic.add_row(row)   

In [ ]:
fig = plt.figure(figsize = (10,10))
plt.scatter(tprfpr_zooniverse_Ic["fpr_zooniverse"], tprfpr_zooniverse_Ic["tpr_zooniverse"], color = "orange")
plt.plot(tprfpr_snid_Ic["fpr_snid"], tprfpr_snid_Ic["tpr_snid"])
plt.title("TPR vs. FPR for different # in agreement Ic")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")

In [ ]:
tprfpr_zooniverse_superfit_Ic = Table(
                names=("phase", "agreement", "tpr_zooniverse", "fpr_zooniverse"
                ),
                meta={"name": "Comparison of Three Programs"},
                dtype=("float64", "float64", "float64", "float64"
                      ))
for phase in phase_cutoffs:
    for agreement in cutoffs:
        truepositive = [0, 0, 0]
        falsepositive = [0, 0, 0]
        for i in joined_phase:
            if(i["phase"] > phase and i["agreement"] >= agreement):
                c_actual = "Ic" in i["Real_c"]
                
                c_zoonivserse = "Ic" in i["Zooniverse_c"]
                c_superfit = "Ic" in i["SF_fit_1"]
                c = c_zoonivserse and c_superfit
                if(c_actual == True and c == True):
                    truepositive[0] += 1
                if(c_actual != True and c == True):
                    falsepositive[0] += 1
        row = [phase, agreement, truepositive[0]/positiveII, falsepositive[0]/negativeII]
        tprfpr_zooniverse_superfit_Ic.add_row(row)

fig = plt.figure(figsize = (10,10))
plt.scatter(tprfpr_zooniverse_superfit_Ic["fpr_zooniverse"], tprfpr_zooniverse_superfit_Ic["tpr_zooniverse"], color = "orange")
plt.plot(tprfpr_snid_Ic["fpr_snid"], tprfpr_snid_Ic["tpr_snid"])
plt.title("TPR vs. FPR for different # in agreement Ic (Superfit and Zooniverse)")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")

In [ ]:
positiveIbc = 0
negativeIbc = 0
for j in joined_phase:
    correct_1a = "Ib" in j["classification"] or "Ic" in j["classification"]
    if(correct_1a):
        positiveIbc += 1
    if(not(correct_1a)):
        negativeIbc += 1

In [ ]:
tprfpr_snid_Ibc = Table(
                names=("rlap", "tpr_snid", "fpr_snid"
                ),
                meta={"name": "Comparison of Three Programs"},
                dtype=("float64", "float64", "float64"
                      )) 
for rlap in np.linspace(0, 25, 251):
    truepositive = [0, 0, 0]
    falsepositive = [0, 0, 0]
    for i in joined_phase:
        if(i["rlap"] >= rlap):
            c_actual = "Ib" in i["Real_c"] or "Ic" in i["Real_c"]
            c_snid = "Ib" in i["Zooniverse_c"] or "Ic" in i["Zooniverse_c"]
            
            if(c_actual == True and c_snid == True):
                truepositive[0] += 1
            if(c_actual != True and c_snid == True):
                falsepositive[0] += 1
    row = [rlap, truepositive[0]/positiveIbc, falsepositive[0]/negativeIbc]
    tprfpr_snid_Ibc.add_row(row)

In [ ]:
tprfpr_zooniverse_Ibc = Table(
                names=("phase", "agreement", "tpr_zooniverse", "fpr_zooniverse"
                ),
                meta={"name": "Comparison of Three Programs"},
                dtype=("float64", "float64", "float64", "float64"
                      )) 
for phase in phase_cutoffs:
    for agreement in cutoffs:
        truepositive = [0, 0, 0]
        falsepositive = [0, 0, 0]
        for i in joined_phase:
            if(i["phase"] > phase and i["agreement"] >= agreement):
                c_actual = "Ib" in i["Real_c"] or "Ic" in i["Real_c"]
                
                c_zoonivserse = "Ib" in i["Zooniverse_c"] or "Ic" in i["Zooniverse_c"]
                c_superfit = "Ib" in i["SF_fit_1"] or "Ic" in i["SF_fit_1"]
                c = c_zoonivserse and c_superfit
                
                if(c_actual == True and c == True):
                    truepositive[0] += 1
                if(c_actual != True and c == True):
                    falsepositive[0] += 1
        row = [phase, agreement, truepositive[0]/positiveIbc, falsepositive[0]/negativeIbc]
        tprfpr_zooniverse_Ibc.add_row(row)   

In [ ]:
fig = plt.figure(figsize = (10,10))
plt.scatter(tprfpr_zooniverse_Ibc["fpr_zooniverse"], tprfpr_zooniverse_Ibc["tpr_zooniverse"], color = "orange")
plt.plot(tprfpr_snid_Ibc["fpr_snid"], tprfpr_snid_Ibc["tpr_snid"])
plt.title("TPR vs. FPR for different # in agreement Ic or Ib (Superfit and Zooniverse)")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.xlim(0, 0.025)
plt.ylim(0, .5)

In [ ]:
is_sedm = []
not_sedm = []
for i in joined_phase["Version"]:
    is_sedm.append("P60" in i)
    not_sedm.append(not("P60" in i))

In [ ]:
negativeIa = 0
positiveIa = 0
for j in joined_phase[is_sedm]:
    correct_1a = "Ia" in j["classification"]
    if(correct_1a):
        positiveIa += 1
    if(not(correct_1a)):
        negativeIa += 1

tprfpr_zooniverse_superfit_Ia = Table(
                names=("phase", "agreement", "tpr_zooniverse", "fpr_zooniverse"
                ),
                meta={"name": "Comparison of Three Programs"},
                dtype=("float64", "float64", "float64", "float64"
                      ))
for phase in phase_cutoffs:
    for agreement in cutoffs:
        truepositive = [0, 0, 0]
        falsepositive = [0, 0, 0]
        for i in joined_phase[is_sedm]:
            if(i["phase"] > phase and i["agreement"] >= agreement):
                c_actual = "Ia" in i["Real_c"]
                
                c_zoonivserse = "Ia" in i["Zooniverse_c"]
                c_superfit = "Ia" in i["SF_fit_1"]
                c = c_zoonivserse and c_superfit
                if(c_actual == True and c == True):
                    truepositive[0] += 1
                if(c_actual != True and c == True):
                    falsepositive[0] += 1
        row = [phase, agreement, truepositive[0]/positiveIa, falsepositive[0]/negativeIa]
        tprfpr_zooniverse_superfit_Ia.add_row(row)

fig = plt.figure(figsize = (10,10))
plt.scatter(tprfpr_zooniverse_superfit_Ia["fpr_zooniverse"], tprfpr_zooniverse_superfit_Ia["tpr_zooniverse"], color = "orange")
plt.plot(tprfpr_snid["fpr_snid"], tprfpr_snid["tpr_snid"])
plt.title("TPR vs. FPR for different # in agreement Ia (Superfit and Zooniverse) SEDM")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")

In [ ]:
negativeIa = 0
positiveIa = 0
for j in joined_phase[not_sedm]:
    correct_1a = "Ia" in j["classification"]
    if(correct_1a):
        positiveIa += 1
    if(not(correct_1a)):
        negativeIa += 1

tprfpr_zooniverse_superfit_Ia = Table(
                names=("phase", "agreement", "tpr_zooniverse", "fpr_zooniverse"
                ),
                meta={"name": "Comparison of Three Programs"},
                dtype=("float64", "float64", "float64", "float64"
                      ))
for phase in phase_cutoffs:
    for agreement in cutoffs:
        truepositive = [0, 0, 0]
        falsepositive = [0, 0, 0]
        for i in joined_phase[not_sedm]:
            if(i["phase"] > phase and i["agreement"] >= agreement):
                c_actual = "Ia" in i["Real_c"]
                
                c_zoonivserse = "Ia" in i["Zooniverse_c"]
                c_superfit = "Ia" in i["SF_fit_1"]
                c = c_zoonivserse and c_superfit
                if(c_actual == True and c == True):
                    truepositive[0] += 1
                if(c_actual != True and c == True):
                    falsepositive[0] += 1
        row = [phase, agreement, truepositive[0]/positiveIa, falsepositive[0]/negativeIa]
        tprfpr_zooniverse_superfit_Ia.add_row(row)

fig = plt.figure(figsize = (10,10))
plt.scatter(tprfpr_zooniverse_superfit_Ia["fpr_zooniverse"], tprfpr_zooniverse_superfit_Ia["tpr_zooniverse"], color = "orange")
plt.plot(tprfpr_snid["fpr_snid"], tprfpr_snid["tpr_snid"])
plt.title("TPR vs. FPR for different # in agreement Ia (Superfit and Zooniverse) NOT SEDM")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")

In [ ]:
tprfpr_zooniverse_superfit_Ia